<a href="https://colab.research.google.com/github/dangdang2222/NLP_basic/blob/main/N_Gram_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U pip
!pip install -U dill
!pip install -U nltk==3.4

In [2]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams

from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten

In [3]:
import nltk
nltk.download("all",quiet=True)

True

In [4]:
text = [['a', 'b', 'c'], ['a', 'c', 'd', 'c', 'e', 'f']]
print(list(bigrams(text[0])))
print(list(ngrams(text[1],n=3)))

[('a', 'b'), ('b', 'c')]
[('a', 'c', 'd'), ('c', 'd', 'c'), ('d', 'c', 'e'), ('c', 'e', 'f')]


## Padding

시작 : \<s>

끝 : \</s>

In [10]:
from nltk.util import pad_sequence

print(list(pad_sequence(text[0],pad_left=True, left_pad_symbol="<s>",
                        pad_right=True, right_pad_symbol="</s>",n=2)))

['<s>', 'a', 'b', 'c', '</s>']


In [12]:
padded_sent = list(pad_sequence(text[0],pad_left=True, left_pad_symbol="<s>",pad_right=True, right_pad_symbol="</s>",n=2))
print(padded_sent)
print(list(ngrams(padded_sent,2)))

['<s>', 'a', 'b', 'c', '</s>']
[('<s>', 'a'), ('a', 'b'), ('b', 'c'), ('c', '</s>')]


In [15]:
from nltk.lm.preprocessing import pad_both_ends

print(list(pad_both_ends(text[1],n=2)))

['<s>', 'a', 'c', 'd', 'c', 'e', 'f', '</s>']


In [17]:
print(list(ngrams(list(pad_both_ends(text[1],n=2)),3)))

[('<s>', 'a', 'c'), ('a', 'c', 'd'), ('c', 'd', 'c'), ('d', 'c', 'e'), ('c', 'e', 'f'), ('e', 'f', '</s>')]


## Everygrams, Flatten

In [20]:
from nltk.util import everygrams

padded_bigrams = list(pad_both_ends(text[0],2)) 
print(padded_bigrams)
print(list(everygrams(padded_bigrams, max_len=3)))

['<s>', 'a', 'b', 'c', '</s>']
[('<s>',), ('a',), ('b',), ('c',), ('</s>',), ('<s>', 'a'), ('a', 'b'), ('b', 'c'), ('c', '</s>'), ('<s>', 'a', 'b'), ('a', 'b', 'c'), ('b', 'c', '</s>')]


In [26]:
from nltk.lm.preprocessing import flatten
print(list(flatten(text)))
print(list(flatten(pad_both_ends(sent, n=2) for sent in text)))


['a', 'b', 'c', 'a', 'c', 'd', 'c', 'e', 'f']
['<s>', 'a', 'b', 'c', '</s>', '<s>', 'a', 'c', 'd', 'c', 'e', 'f', '</s>']


In [33]:
from nltk.lm.preprocessing import padded_everygram_pipeline
training_ngrams, padded_sentences = padded_everygram_pipeline(2, text)
# n차원만큼 padd + max_len=n인 everygram
# padded_sentence = flatten + pad

for ngram in training_ngrams:
  print(list(ngram))

print(list(padded_sentences))

[('<s>',), ('a',), ('b',), ('c',), ('</s>',), ('<s>', 'a'), ('a', 'b'), ('b', 'c'), ('c', '</s>')]
[('<s>',), ('a',), ('c',), ('d',), ('c',), ('e',), ('f',), ('</s>',), ('<s>', 'a'), ('a', 'c'), ('c', 'd'), ('d', 'c'), ('c', 'e'), ('e', 'f'), ('f', '</s>')]
['<s>', 'a', 'b', 'c', '</s>', '<s>', 'a', 'c', 'd', 'c', 'e', 'f', '</s>']


## Exercise

In [34]:
import os
import requests
import io

# Text version of https://kilgarriff.co.uk/Publications/2005-K-lineer.pdf
if os.path.isfile('language-never-random.txt'):
    with io.open('language-never-random.txt', encoding='utf8') as fin:
        text = fin.read()
else:
    url = "https://gist.githubusercontent.com/alvations/53b01e4076573fea47c6057120bb017a/raw/b01ff96a5f76848450e648f35da6497ca9454e4a/language-never-random.txt"
    text = requests.get(url).content.decode('utf8')
    with io.open('language-never-random.txt', 'w', encoding='utf8') as fout:
        fout.write(text)

In [ ]:
text

In [53]:
from nltk import word_tokenize, sent_tokenize

sentences = sent_tokenize(text)
tokenized_text = [list(map(str.lower,word_tokenize(sent))) for sent in sentences]

In [54]:
tokenized_text[0]

['language',
 'is',
 'never',
 ',',
 'ever',
 ',',
 'ever',
 ',',
 'random',
 'adam',
 'kilgarriff',
 'abstract',
 'language',
 'users',
 'never',
 'choose',
 'words',
 'randomly',
 ',',
 'and',
 'language',
 'is',
 'essentially',
 'non-random',
 '.']

padding이나 ngram을 사용하려면 먼저 tokenizing해야함

In [55]:
n=3
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

#train_data는 n차원만큼 padd+everygram
#padded_sents는 padd+flatten

In [56]:
from nltk.lm import MLE
model = MLE(n) #n은 ngram 차원

In [57]:
len(model.vocab)

0

In [58]:
model.fit(train_data, padded_sents)

In [59]:
len(model.vocab)

1391

In [60]:
print(model.vocab.lookup(tokenized_text[0]))

('language', 'is', 'never', ',', 'ever', ',', 'ever', ',', 'random', 'adam', 'kilgarriff', 'abstract', 'language', 'users', 'never', 'choose', 'words', 'randomly', ',', 'and', 'language', 'is', 'essentially', 'non-random', '.')


In [67]:
# 만약 Vocab 집합에 포함되지 않는 단어라면 <UNK>라는 특수 토큰으로 처리된다.
print('language is never random lol .'.split())
print(model.vocab.lookup('language is never random lol .'))
print(model.vocab.lookup('language is never random lol .'.split()))

['language', 'is', 'never', 'random', 'lol', '.']
<UNK>
('language', 'is', 'never', 'random', '<UNK>', '.')


In [68]:
# count
print(model.counts)
model.counts['language']

<NgramCounter with 3 ngram orders and 19611 ngrams>


25

In [71]:
print(model.counts[['language']]['is']) #Count('is'|'language')
print(model.counts[['language','is']]['never']) #Count('never'|'language is')

11
7


In [72]:
# score
print(model.score('language'))
print(model.score('is', 'language'.split()))  # P('is'|'language')
print(model.score('never', 'language is'.split()))  # P('never'|'language is')

0.003691671588895452
0.44
0.6363636363636364


In [74]:
print(model.score("<UNK>"))
print(model.score("lah"))
print(model.score("<UNK>")==model.score("lah"))

0.0
0.0
True


In [75]:
model.logscore("never","language".split())

-inf

In [76]:
model.logscore("never","language is".split())

-0.6520766965796932

## N-gram으로 새로운 텍스트 생성

In [78]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=42):
  content=[]
  for token in model.generate(num_words, random_seed = random_seed):
    if token=='<s>':
      continue
    if token=='</s>':
      break
    content.append(token)
  return detokenize(content)

In [80]:
print(model.generate(20, random_seed=7))
generate_sent(model,20,random_seed=7)

['and', 'hence', ',', 'when', 'we', 'look', 'at', 'linguistic', 'phenom-', 'ena', 'in', 'corpora', ',', '216⫺223', '.', '</s>', '</s>', '</s>', '</s>', '</s>']


'and hence, when we look at linguistic phenom- ena in corpora , 216⫺223.'

In [81]:
print(model.generate(30, random_seed=10))
print(generate_sent(model,30,random_seed=10))

['is', 'not', 'possible', 'at', 'all', 'in', 'the', 'last', 'paragraph', 'is', '‘', 'indistinguishable', '’', '.', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']
is not possible at all in the last paragraph is ‘ indistinguishable ’.
